In [13]:
import numpy as np

def gauss_jordan(matrix): #calcula a inversa
    rows, cols = matrix.shape
    identity = np.identity(rows)
    augmented_matrix = np.concatenate((matrix, identity), axis=1)

    for i in range(rows):
        pivot_row = i
        while pivot_row < rows and augmented_matrix[pivot_row, i] == 0:
            pivot_row += 1
        if pivot_row == rows:
            raise ValueError("Nao tem inversa")
        augmented_matrix[[i, pivot_row]] = augmented_matrix[[pivot_row, i]]
        pivot = augmented_matrix[i, i]
        augmented_matrix[i] /= pivot
        for j in range(rows):
            if j != i:
                factor = augmented_matrix[j, i]
                augmented_matrix[j] -= factor * augmented_matrix[i]
    return augmented_matrix[:, cols:]

def matrix_multiply(A, B): #multiplicacao entre matriz
    rows_A, cols_A = A.shape
    rows_B, cols_B = B.shape

    if cols_A != rows_B:
        raise ValueError("Número de colunas da primeira matriz deve ser igual ao número de linhas da segunda matriz.")

    # Inicializar a matriz resultado com zeros
    result = np.zeros((rows_A, cols_B))

    # Realizar a multiplicação de matrizes
    for i in range(rows_A):
        for j in range(cols_B):
            for k in range(cols_A):
                result[i, j] += A[i, k] * B[k, j]

    return result

def simplex_method(c, A, b, maximize=True):
    # Se for maximização, multiplicar a função objetivo por (-1) para converter em minimização
    if maximize:
        c = -c

    # Verificar e ajustar restrições com b_i < 0 (Parte da Fase I, Passo 2)
    for i in range(len(b)):
        if b[i] < 0:
            A[i] = -A[i]
            b[i] = -b[i]

    # Adicionar variáveis de folga (Parte da Fase I, Passo 2)
    num_constraints, num_vars = A.shape
    slack_vars = np.eye(num_constraints)
    A = np.hstack([A, slack_vars])

    # Função objetivo com variáveis de folga (Parte da Fase I, Passo 2)
    c = np.hstack([c, np.zeros(num_constraints)])

    # Inicializar a base (Parte da Fase I, Passo 2)
    basic_vars = list(range(num_vars, num_vars + num_constraints))
    non_basic_vars = list(range(num_vars))

    # Iterar até encontrar a solução ótima (Parte da Fase I e Fase II)
    while True:

        B = A[:, basic_vars] #anexa a matriz basica
        print("Matriz básicas:")
        print(B)

        N = A[:, non_basic_vars] #nb
        print("Matriz não básica:")
        print(N)

        c_B = c[basic_vars]
        print("Custos básicos:")
        print(c_B)

        c_N = c[non_basic_vars]
        print("Custos não básicos:")
        print(c_N)

        B_inv = gauss_jordan(B) #inversa
        print("B inversa: ")
        print(B_inv)

        x_B = matrix_multiply(B_inv, b.reshape(-1, 1)).flatten() #passo 1 calculo da solucao basica
        print("Solucao basica x_B:")
        print(x_B)

        # Passo 2: calcular custos relativos (tanto fas e I e II)
        #2.1

        lambda_T = matrix_multiply(c_B.reshape(1, -1), B_inv).flatten()
        print("Lambda_T:")
        print(lambda_T)

        #2.2

        reduced_costs = c_N - matrix_multiply(lambda_T.reshape(1, -1), N).flatten()
        print("Custos relativos:")
        print(reduced_costs)

        #3 otimalidade
        # Teste de otimalidade Fase I se cnk >=0 entao
        print("Se todos os custos são >= 0")
        if all(rc >= 0 for rc in reduced_costs): #verifica se todos os custos sao positivos
            solution = np.zeros(len(c)) #cria um vetor do tamanho de c comm zeros 0
            print("Solução, solution: ")
            print(solution)
            for i, var in enumerate(basic_vars):
                solution[var] = x_B[i] #Preenche o vetor solution com os valores das variáveis básicas da solução atual (atualiza a solucao otima, com os novos valores das var basicas)
            print("Solution preenchida: ")
            print(solution)
            optimal_value = np.dot(c, solution) #calulca na funcao obj: o valor ótimo multiplicando a função objetivo c pela solução .
            print("Valor ótimo:")
            print(optimal_value)
            print("Se for maximizar, inverta o sinal")
            if maximize:
                optimal_value = -optimal_value #transforma pra max dnv
            print("Vetor solucao inteiro: ")
            print(solution)
            print("Vetor solucao particionado: ")
            print(solution[:num_vars])
            print("Valor ótimo: ")
            print(optimal_value)
            return solution[:num_vars], optimal_value  #retorna a solução ótima e o valor ótimo

        # Determinar a variável a entrar na base (Parte da Fase I e Fase II)
        entering_var = non_basic_vars[np.argmin(reduced_costs)]
        print("Non_basic_vars: ")
        print(non_basic_vars)
        print("np.argmin(reduced_costs)")
        print(np.argmin(reduced_costs))
        print("Variável a entrar na base: ")
        print(entering_var)

        # Calcular direção simplex (Parte da Fase I e Fase II)
        print("Entering_var")
        print(entering_var)
        print(A[:,entering_var])
        y = matrix_multiply(B_inv, A[:, entering_var].reshape(-1, 1)).flatten()
        print("y")
        print(y)

        print("Se y<=0 é problema ilimitado")
        # se todas as razoes forem negativas ou 0, n da
        if all(y <= 0):
            raise Exception("Problema ilimpitado")
        #2.3
        # Determinar a variável a sair da base (fase 1 e 2)
        ratios = np.array([x_B[i] / y[i] if y[i] > 0 else np.inf for i in range(len(y))]) #lista de valores minimos para cada indice i onde o y é positivo
        print("ratios:")
        print(ratios)
        leaving_var_index = np.argmin(ratios) #determina qual indice da variavel que deve sair da base (np.argmin retorna o menor valor)
        print("leaving_var_index:")
        print(leaving_var_index)
        leaving_var = basic_vars[leaving_var_index] #retirando do vetor das var basicas o menor valor
        print("leaving_var:")
        print(leaving_var)

        # att as variáveis básicas e não-básicas (fase 1 e 2)
        basic_vars[leaving_var_index] = entering_var #att a base que ta saindo pela que entrando (nova basee)
        print("basic_vars")
        print(basic_vars)
        non_basic_vars[non_basic_vars.index(entering_var)] = leaving_var #att as var nao basica
        print("non_basic_vars")
        print(non_basic_vars)

# Testar com os problemas fornecidos

# Problema  #max
b = np.loadtxt('b.txt')
c = np.loadtxt('c.txt')
A = np.loadtxt('A.txt') 
print(b)
print(c)
print(A)

solution, optimal_value = simplex_method(c, A, b, maximize=True)
print("Problema 1):")
print("Solução:", solution)
print("Valor ótimo:", optimal_value)

# print("Ignora")
# print("Ignora")
# print("Ignora")
# print("Ignora")

# # Problema #min
# c = np.array([4, -12])
# A = np.array([[2, 1], [1, 3],[1,0]])
# b = np.array([6, 8,4])

# solution, optimal_value = simplex_method(c, A, b, maximize=False)
# print("Problema 2):")
# print("Solução:", solution)
# print("Valor ótimo:", optimal_value)
#arrumar a fase 1 do menor igual e string


[]
[6. 2.]
[[3. 1.]
 [1. 1.]]
Matriz básicas:
[[1. 0.]
 [0. 1.]]
Matriz não básica:
[[3. 1.]
 [1. 1.]]
Custos básicos:
[0. 0.]
Custos não básicos:
[-6. -2.]
B inversa: 
[[1. 0.]
 [0. 1.]]


C:\Users\yorra\AppData\Local\Temp\ipykernel_155672\2521537495.py:173: UserWarning: loadtxt: input contained no data: "b.txt"
  b = np.loadtxt('b.txt')


ValueError: Número de colunas da primeira matriz deve ser igual ao número de linhas da segunda matriz.